In [4]:
from snowflake.snowpark import Session

In [3]:
!pip show snowflake.

In [2]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()
print(session)

<snowflake.snowpark.session.Session: account="FYA62509", role="PRAKHARGOYAL", database="INSIGHT_DESIGNER_SPCS", schema="INSIGHT_SPCS_SCHEMA", warehouse="FOSFOR_INSIGHT_WH">


In [3]:
from fosforml.model_manager import snowflakesession
session_instance = snowflakesession()
params = session_instance.connection_params
print(params)

{'host': 'snowflake.us-east-1.snowflakecomputing.com', 'port': '443', 'protocol': 'https', 'account': 'FYA62509', 'authenticator': 'oauth', 'token': 'ver:1-hint:32355341568585746-ETMsDgAAAZDpOaTgABRBRVMvQ0JDL1BLQ1M1UGFkZGluZwEAABAAEGMOB6MAYnwhV5h5GcfFYhsAAACQIuKj6nKgp2ZTvcP87QnJ2qmzT78Oy7AlsRfExRl+Pvb/01n+9kpq+Rh/04qWD4zQF680bfuU/xx7piSRzEr6U2lt2OBaCCv1ifgOv472G8d501WI+R7XPLGdOEebWVFprSkhvaQsX0nWgel3oOywtfJ4SkAhZ+UDHwWEIZskuLjVargq7iK5SKmXEv/atMToABR51ZmrsmmJmdmMu+BsHVg19Rllgg==', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'INSIGHT_DESIGNER_SPCS', 'schema': 'INSIGHT_SPCS_SCHEMA', 'client_session_keep_alive': True}


#### Snowflake Modeling

In [4]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [5]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [6]:
# snow_df = session.table("HR_CHURN")
from fosforml.model_manager.snowflakesession import get_session
session = get_session()
import pandas as pd
df =pd.read_csv("HR_CHURN.csv")
snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [7]:
input_data_frame = input_data_frame.drop("Unnamed: 0")
test_df = test_df.drop("Unnamed: 0")

In [8]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [9]:
input_data_frame.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |
|2.0       |7.0           |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |0.72              

In [10]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

The version of package 'snowflake-snowpark-python' in the local environment is 1.20.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [11]:
test_df = apply_label_encoding(test_df)

In [12]:
pred_df = pipeline.predict(test_df)

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


#### Model Registration,

In [13]:
from fosforml import register_model

In [14]:
register_model(
  model_obj=pipeline,
  session=session,
  name="SAURABH_HR_CHURN_25July_1",
  snowflake_df=pred_df,
  dataset_name="HR_CHURN",
  dataset_source="SnowflakeDataset",
  description="This is a test model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
Failed to set tags for model MODEL_4A484B6E_510B_4E0D_BFF1_DC5A3413E908_FDC_SAURABH_HR_CHURN_25JULY.
(1300) (1304): 01b5e6e0-0607-8707-0072-f3030fa2a8ea: 003001 (42501): SQL access control error:
Insufficient privileges to operate on tag 'FLAVOR'


"Model 'MODEL_4A484B6E_510B_4E0D_BFF1_DC5A3413E908_FDC_SAURABH_HR_CHURN_25JULY' registered successfully."